## Cleaning & Standardizing 10k Iranian Dirty Transactions Dataset

In [1]:
import pandas as pd

# Ganti path sesuai lokasi file kamu
df = pd.read_csv('archive/trx-10k.csv')

# Cek 5 data teratas
print("--- 5 Baris Pertama ---")
display(df.head())

# Cek tipe data dan memori
print("\n--- Info Tipe Data ---")
print(df.info())

--- 5 Baris Pertama ---


,status,time,card_type,city,amount,id
0,success,2025-09-07 10:48:00,Visa,Tehran,1427657.0,98
1,success,2025-09-20 13:10:00,MastCard,Tehran,1578078.0,52
2,fail,2025-09-29 03:27:00,Visa,Tehran,250000.0,20
3,success,2025-09-15 13:13:00,Discover,Isfahan,0.0,40
4,success,2025-09-11 11:11:00,Visa,Tehran,1782689.0,59



--- Info Tipe Data ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   status     10000 non-null  object 
 1   time       10000 non-null  object 
 2   card_type  9689 non-null   object 
 3   city       9884 non-null   object 
 4   amount     10000 non-null  float64
 5   id         10000 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 468.9+ KB
None


In [2]:
# Hitung jumlah null
null_counts = df.isnull().sum()

# Hitung persentase
null_pct = (null_counts / len(df)) * 100

# Gabungkan jadi dataframe biar enak dilihat
null_summary = pd.DataFrame({
    'Jumlah Null': null_counts,
    'Persentase (%)': null_pct
})

# Tampilkan hanya kolom yang ada null-nya, urutkan dari yang terbesar
print("\n--- Summary Missing Values ---")
print(null_summary[null_summary['Jumlah Null'] > 0].sort_values('Persentase (%)', ascending=False))


--- Summary Missing Values ---
           Jumlah Null  Persentase (%)
card_type          311            3.11
city               116            1.16
